# Gradient boosting on mhcflurry dataset

## Setup and data loading

In [15]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Embedding, LSTM, Merge, Flatten

import xgboost as xgb

import sklearn.cross_validation as xval

sns.set(style='whitegrid', font_scale=1.2, rc={'figure.figsize': (10, 8)})

In [2]:
rdata = pd.read_csv('../data/flurry/A0201.csv')

In [3]:
rdata.head()

,sequence,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,AAAFVNQHL,9,5011.872336,10889.300933,16069.412530,6576.578374
1,AAAQGQAPL,9,19998.618696,5701.642723,12852.866599,3006.076303
2,AADSFATSY,9,19998.618696,28575.905434,24831.331053,92257.142715
3,AALGLWLSV,9,2999.162519,376.703799,187.068214,406.443329
4,AALQGGGPP,9,65012.969034,30902.954325,41304.750199,495450.190805


In [4]:
rdata.describe()

,length,meas,netmhc,netmhcpan,smmpmbec_cpp
count,2126.000000,2126.000000,2126.000000,2126.000000,2.126000e+03
mean,9.163688,10823.129166,7894.649351,9120.768101,4.622073e+05
std,0.370079,21356.218252,11775.998358,14194.733557,1.088238e+07
min,9.000000,1.030386,2.398833,1.039920,3.443499e-02
25%,9.000000,16.060237,26.136668,22.672576,6.338697e+01
50%,9.000000,254.977930,159.587915,193.865392,2.200393e+02
75%,9.000000,19998.618696,15126.909749,15559.656316,7.529224e+03
max,10.000000,250034.536170,34914.031548,48977.881937,4.965923e+08


## Pad protein strings to their max length

In [5]:
PROTEIN_ALPHABET = 'ACDEFGHIKLMNPQRSTVWY'
PROTEIN_PLACEHOLDER = 'Z'
PROTEIN_LETTERS = PROTEIN_ALPHABET + PROTEIN_PLACEHOLDER

In [6]:
rdata.length.value_counts()

9     1778
10     348
Name: length, dtype: int64

In [11]:
seq_maxlen = max(rdata.length)
rdata.sequence = rdata.sequence.apply(lambda seq: seq.ljust(seq_maxlen, PROTEIN_PLACEHOLDER))

## Create one-hot-encoded features for each aminoacid-position pair

In [1]:
train_features = []
for pos in range(seq_maxlen):
    colname_prefix = 'p' + str(pos + 1).zfill(len(str(seq_maxlen)))
    for aminoacid in PROTEIN_LETTERS:
        colname = colname_prefix + aminoacid
        train_features.append(colname)
        rdata[colname] = rdata.sequence.apply(lambda seq: float(seq[pos] == aminoacid))

NameError: name 'seq_maxlen' is not defined

In [40]:
rdata.head()

,sequence,length,meas,netmhc,netmhcpan,smmpmbec_cpp,p01A,p01C,p01D,p01E,...,p10N,p10P,p10Q,p10R,p10S,p10T,p10V,p10W,p10Y,p10Z
0,AAAFVNQHLZ,9,5011.872336,10889.300933,16069.412530,6576.578374,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,AAAQGQAPLZ,9,19998.618696,5701.642723,12852.866599,3006.076303,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,AADSFATSYZ,9,19998.618696,28575.905434,24831.331053,92257.142715,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,AALGLWLSVZ,9,2999.162519,376.703799,187.068214,406.443329,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,AALQGGGPPZ,9,65012.969034,30902.954325,41304.750199,495450.190805,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Train XGBoost

In [41]:
X = rdata[train_features].values
y = rdata.meas.values

In [55]:
X_train, X_test, y_train, y_test = xval.train_test_split(X, y)

In [56]:
dm_train = xgb.DMatrix(X_train, y_train)
dm_test = xgb.DMatrix(X_test, y_test)

In [57]:
params = {}

In [58]:
xgb.cv(params, dm_train)

/Users/emil/local/anaconda/lib/python3.5/site-packages/xgboost/training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,22088.768880,654.725385,21081.603516,184.129734
1,20664.921224,578.209600,18567.916016,165.774677
2,19984.945313,556.939192,16743.243490,123.476008
3,19413.001953,372.922617,15235.276693,124.011766
4,19126.108724,310.861331,14120.229818,129.709583
5,18958.535807,351.960423,13295.804688,180.485035
6,18787.201172,419.906664,12633.383138,190.312046
7,18761.882812,439.612182,12044.685547,215.203426
8,18723.759114,503.009362,11547.236654,235.131129
9,18776.842448,540.739999,10979.814127,316.627316


## Convert protein string into array of integers

In [11]:
# PROTEIN_LETTERS_INDICES = {c: PROTEIN_LETTERS.find(c) + 1 for c in PROTEIN_LETTERS}

In [12]:
# X = np.array(list(rdata.sequence.apply(lambda seq: np.array([PROTEIN_LETTERS_INDICES[c] for c in seq]))))

In [13]:
# Y = rdata.meas.values

## Split the data into train/test sets.

In [14]:
# X_train, X_test, Y_train, Y_test = xval.train_test_split(X, Y)

## Prepare two-way LSTM models

In [37]:
# def twoway(x):
#     return [x, np.fliplr(x)]

In [50]:
# def branch_model():
#     return Sequential([
#             Embedding(input_dim=len(PROTEIN_LETTERS) + 1, output_dim=128, input_length=seq_maxlen),
#             LSTM(10),
#         ])

# ltr_model = branch_model()
# rtl_model = branch_model()
# model = Sequential([
#     Merge([ltr_model, rtl_model], mode='sum'),
#     Dense(32, activation='tanh'),
#     Dense(8, activation='tanh'),
#     Dense(1),
# ])

In [55]:
# model.compile(optimizer='rmsprop', loss='mae')

In [56]:
# model.fit(twoway(X_train), Y_train, batch_size=16, nb_epoch=10);

Epoch 1/10
1594/1594 [==============================] - 0s - loss: 10820.1268     
Epoch 2/10
1594/1594 [==============================] - 1s - loss: 10819.6996     
Epoch 3/10
1594/1594 [==============================] - 0s - loss: 10819.2760     
Epoch 4/10
1594/1594 [==============================] - 0s - loss: 10818.8474     
Epoch 5/10
1594/1594 [==============================] - 1s - loss: 10818.4566     
Epoch 6/10
1594/1594 [==============================] - 0s - loss: 10818.0527     
Epoch 7/10
1594/1594 [==============================] - 0s - loss: 10817.6729     
Epoch 8/10
1594/1594 [==============================] - 1s - loss: 10817.3061     
Epoch 9/10
1594/1594 [==============================] - 1s - loss: 10816.9306     
Epoch 10/10
1594/1594 [==============================] - 1s - loss: 10816.5771     


In [48]:
# score = model.evaluate(twoway(X_test), Y_test, batch_size=16)
# print()
# print(score)

512/532 [===========================>..] - ETA: 0s
10794.3852869


In [57]:
# model.predict(twoway(X_test))

array([[ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21.19623947],
       [ 21